<a href="https://colab.research.google.com/github/kckohkoh2/fantastic-disco/blob/master/day0412.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(1) 설치하기

In [ ]:
!apt-get update #apt-get은 리눅스 시스템 패키지를 관리하는 명령어

In [ ]:
!pip install mujoco #설치하기

코랩에는 GUI가 따로 없기 때문에 MuJoCo 환경을 렌더링하기 위해서는 pyvirtualdisplay가 필요하다. pyvirtualdisplay를 사용하기 위해서는 Xvfb (X virtual framebuffer) 파일이 필요하므로 xvfb와 xvfbwrapper를 추가로 설치

In [ ]:
!pip install pyvirtualdisplay
!sudo apt-get install xvfb
!pip install xvfbwrapper

(6-1) 모듈 및 라이브러리 import

gym: OpenAI Gym의 강화학습 환경을 사용하기 위한 라이브러리

In [ ]:
import gym

glob: 특정 경로의 파일 리스트를 가져올 수 있는 파이썬 표준 모듈입니다.
예: glob.glob("video/*.mp4")로 video 폴더 내의 mp4 파일들을 모두 찾음.

In [ ]:
import glob

io: 파일을 바이너리로 읽기 위한 입출력 라이브러리입니다.
비디오 파일을 바이트로 읽기 위해 사용됩니다.

In [ ]:
import io

base64: 비디오 데이터를 base64 인코딩하여 웹에서 표시할 수 있게 합니다.
Jupyter 노트북에서 비디오를 직접 삽입하려면 base64 인코딩이 필요합니다.
IPython.display: Jupyter Notebook에서 HTML이나 비디오를 표시하기 위한 모듈입니다.

In [ ]:
import base64

수치 연산용 패키지로, 상태/행동/보상 등을 배열로 처리하는 데 자주 사용됩니다.
예: np.array(action), np.mean() 등.

In [ ]:
!pip install numpy==1.23.5
import numpy as np
import torch

Jupyter 노트북 내에서 HTML 기반의 영상 또는 출력 삽입을 위해 사용됩니다.
ipythondisplay.display(HTML(...)) 형태로 비디오 렌더링.

In [ ]:
from IPython.display import HTML
from IPython import display as ipythondisplay

RecordVideo: Gym 환경의 비디오를 자동 녹화해주는 래퍼입니다.
Gym 환경을 비디오로 저장하기 위한 비디오 래퍼입니다.
환경을 감싸면 자동으로 .mp4 파일이 생성됩니다.

In [ ]:
from gym.wrappers.record_video import RecordVideo

pyvirtualdisplay: GUI가 없는 환경(예: Colab)에서도 렌더링이 가능하게 가상 디스플레이를 설정하는 도구입니다.
Display(visible=0, size=(1400, 900)) 설정 후 .start() 하면 사용 가능.

In [ ]:
from pyvirtualdisplay import Display

(6-2) 화면을 가상으로 생성하는 설정

Display는 pyvirtualdisplay 모듈의 클래스입니다.
visible=0: 실제 화면을 표시하지 않음 (백그라운드에서 실행).
size=(1400, 900): 가상 디스플레이 해상도 설정.
즉, GUI가 없는 환경(예: Google Colab, 리눅스 서버)에서도 렌더링이 가능한 화면을 가상으로 생성하는 설정입니다.

In [ ]:
display = Display(visible=0, size=(1400, 900))
display.start()

(6-3) Google Colab에서 Gym 환경의 실행 결과로 저장된 .mp4 비디오 파일을 직접 노트북 셀에 표시하는 함수

In [ ]:
#영상 표시 함수
def show_video():
    mp4list = glob.glob('video/*.mp4') #'video'폴더에서 mp4 확장자를 가진 모든 파일 경로를 리스트로 가져옴
    if len(mp4list) > 0:   #mp4 파일이 하나 이상 있을 경우에만 다음 처리를 합니다.
        mp4 = mp4list[0]   #mp4 파일 중 첫 번째 파일을 열고
        video = io.open(mp4, 'r+b').read() #(r+b: 읽기 + 바이너리 모드) 내용을 바이트 단위로 읽습니다.
        encoded = base64.b64encode(video) #읽은 비디오 데이터를 base64 형식으로 인코딩
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

(6-4) 클래스 DummyAgent

이름을 Dummy로 지은 것은 강화학습을 실제로 수행하지 않고도 환경을 실행해보기 위한 가짜 에이전트 클래스이기 때문입니다.

표준편차가 0.5인 8차원 벡터를 생성합니다.
이는 샘플링할 때 행동이 mu=0을 중심으로 분산되도록 설정함을 의미합니다.
Normal(mu, sigma) 분포에서 행동을 샘플링할 수 있게 해줍니다.


In [ ]:
# 더미 에이전트 정의
class DummyAgent:
    def get_action(self, state):   #주어진 상태(state)를 입력받아, 행동 분포를 반환
        mu = torch.zeros(8).to(state.device)         # Ant-v4의 액션 공간은 8차원 벡터(평균 0)
        sigma = torch.ones(8).to(state.device) * 0.5 #확률적 정책(표준편차 0.5)
        return mu, sigma

# 에이전트 인스턴스 생성
agent = DummyAgent()

(6-5) 설정 및 초기화

device는 다음과 같이 정의될 수 있습니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
결론: to(state.device)는 GPU/CPU 모두 자동 대응되는 똑똑한 장치 선택자입니다.

In [ ]:
# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'Ant-v4': MuJoCo 기반의 네 다리 로봇 걷기 환경입니다.
render_mode="rgb_array": 화면을 numpy 배열(RGB 이미지)로 렌더링하게 만듭니다.Ant-v4는 MuJoCo 설치가 필요합니다 (pip install mujoco gym[mujoco]).

In [ ]:
# 환경 생성 및 설정
env_name = 'Ant-v4'
env = gym.make(env_name, render_mode="rgb_array")
env = RecordVideo(env, './video', episode_trigger=lambda episode_number: True)

(6-6) 에피소드 실행
Gym 환경의 한 에피소드를 실행하며, 에이전트가 행동을 선택하고, 환경에 적용하는 강화학습 시뮬레이션 루프입니다.

15초정도 걸림

In [ ]:
# 환경 초기화
state = env.reset()
# 에피소드 실행
while True:
    env.render() #현재 환경 상태를 화면에 렌더링합니다.

    # 상태를 텐서로 변환하여 에이전트에 전달
    state_tensor = torch.from_numpy(state).float().to(device) #2. 상태 → 텐서 변환 (GPU로 이동 포함)
    mu, sigma = agent.get_action(state_tensor) #에이전트로부터 행동 분포 얻기
    dist = torch.distributions.Normal(mu, sigma) #평균과 분산으로부터 정규분포 생성
    action = dist.sample().cpu().numpy() #.sample()로 행동 샘플링 (확률적),다시 넘파이 배열로 변환
    # 환경에 액션 적용
    state, reward, done, info = env.step(action) #환경에 action 적용 → 다음 상태 반환
    if done:
        break #done == True면 에피소드 종료

In [ ]:
# 환경 종료 및 영상 표시
env.close()
show_video()

In [ ]:
import gym
import glob
import io
import base64
import torch
import numpy as np
from IPython.display import HTML
from IPython import display as ipythondisplay
from gym.wrappers.record_video import RecordVideo
from pyvirtualdisplay import Display

# 가상 디스플레이 설정 (Colab 환경에서 필요)
display = Display(visible=0, size=(1400, 900))
display.start()

# 영상 표시 함수
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

# 더미 에이전트 정의
class DummyAgent:
    def get_action(self, state):
        mu = torch.zeros(8).to(state.device)       # Ant-v4의 액션 공간은 8차원
        sigma = torch.ones(8).to(state.device) * 0.5
        return mu, sigma

# 에이전트 인스턴스 생성
agent = DummyAgent()

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 환경 생성 및 설정
env_name = 'Ant-v4'
env = gym.make(env_name, render_mode="rgb_array")
env = RecordVideo(env, './video', episode_trigger=lambda episode_number: True)

# 환경 초기화
state = env.reset()

# 에피소드 실행
while True:
    env.render()

    # 상태를 텐서로 변환하여 에이전트에 전달
    state_tensor = torch.from_numpy(state).float().to(device)
    mu, sigma = agent.get_action(state_tensor)
    dist = torch.distributions.Normal(mu, sigma)
    action = dist.sample().cpu().numpy()

    # 환경에 액션 적용
    state, reward, done, info = env.step(action)

    if done:
        break

# 환경 종료 및 영상 표시
env.close()
show_video()